<h1>Hindustan Uniliver Hackathon</h1>

<h3>Importing useful libraries and data</h3>

In [1]:
#Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_excel("Sales_Services_July 2019.xlsx")

In [3]:
data_original = data

<h3>Converting to dates to pandas datetime format</h3>

In [4]:
data['Sold On']=pd.to_datetime(data['Sold On'],format='%Y-%m-%d')

In [5]:
data['Serviced On']=pd.to_datetime(data['Serviced On'],format='%Y-%m-%d')

In [6]:
data['Day']=pd.DatetimeIndex(data['Closed On']).day
#data['Day']= pd.DatetimeIndex(data['Serviced On']).day

In [7]:
data['Day']=data['Day'].astype(str)

<h3> Feature Engineering and Generation</h3>

In [8]:
#Payment Type
p = pd.DataFrame(data['Payment Type'].unique(), columns={'Payment Type'})
data['Payment Type'].fillna('-999', inplace=True)

In [9]:
# The payment column had a lot of unique values. Dropping this coliumn didn't make sense as it was important for 
# anomayly detection.Therefore we tried to shrink the number of variables to broader categories.
payment_method = []
methods = ['Cash', 'Visa', 'Mastercard', 'Debit Card']
methods1 = ['Membership', 'Custom', 'coupon']
for i in data['Payment Type']:
    if i=='Cash':
        payment_method.append('Cash')
    elif i=='Visa':
        payment_method.append('Visa')
    elif i=='Mastercard':
        payment_method.append('Mastercard')
    elif i=='Debit Card':
        payment_method.append('Debit Card')
    elif i == '-999':
        payment_method.append('Unknown')
    else:
        flag_mem = 0
        flag_custom = 0
        flag_coupon = 0
        flag_other = 0
        l = list(i.split())
        for j in range(len(l)):
            if l[j] in methods:
                flag_other = 1
            if l[j]=='Custom':
                flag_custom = 1
            if l[j]=='coupon':
                flag_coupon = 1
            if l[j]=='Membership':
                flag_mem = 1
        if flag_other == 1 and (flag_coupon == 1 or flag_custom == 1 or flag_mem == 1):
            payment_method.append('Double')
        elif flag_coupon == 1 and flag_other == 0:
            payment_method.append('Coupon')
        elif flag_custom == 1 and flag_other == 0:
            payment_method.append('Custom')
        elif flag_mem == 1 and flag_other == 0:
            payment_method.append('Membership')
        else:
            payment_method.append('Others')
#     elif 'Membership' in str(i):
#         payment_method.append('Membership')
#     elif 'Custom' in str(i):
#         payment_method.append('Custom')
#     elif i.isna():
#         payment_method.append('None')
#     else:
#         payment_method.append('Double')


In [10]:
data['Store_Day']=data['Center Name']+"_"+(data['Day'])

In [11]:
#Total Number of services daywise in a store
groupbytot=data.groupby('Store_Day')['ServiceName'].count()
data['total_Serv']=data['Store_Day'].map(groupbytot)

In [12]:
#Total Number of services avg monthly in a store
groupbyavg=data.groupby('Center Name')['ServiceName'].count()
data['Avg_Serv']=data['Center Name'].map(groupbyavg)
data['Avg_Serv']=data['Avg_Serv']/31

In [14]:
#Total No of Invoices in a store in a day
groupbytot=data.groupby('Store_Day')['Invoice No'].nunique()
data['totalInvo/Day']=data['Store_Day'].map(groupbytot)

In [15]:
#Total No of Invoices in a store in a month
groupbytot=data.groupby('Center Name')['Invoice No'].nunique()
data['totalInvo/Month']=data['Center Name'].map(groupbytot)

In [16]:
data['NetPrice0']=(data['Net Price']==0)

In [17]:
data['NetPrice0']=data['NetPrice0'].astype(int)

In [18]:
#Total No of NetPrice Zeros in a store
groupbytot=data.groupby('Center Name')['NetPrice0'].sum()
data['NetPrice0']=data['Center Name'].map(groupbytot)

In [19]:
data['Invoice No']=data['Invoice No'].astype(str)

In [20]:
#Store_Invoice_No_Day
data['Store_Invoice No_Day']=data['Center Name']+"_"+(data['Invoice No'])+"_"+(data['Day'])

In [21]:
#Total Number of Unique Service Invoices in a store day wise
groupbytot=data.groupby('Store_Invoice No_Day')['ServiceName'].nunique()
data['UniqueService/InvoiceNo/Day/Store']=data['Store_Invoice No_Day'].map(groupbytot)

In [22]:
#Total Number of Repeating Invoices in a store day wise
groupbytot=data.groupby('Store_Invoice No_Day')['Level'].count()
data['InvoiceNo/Day/Store']=data['Store_Invoice No_Day'].map(groupbytot)

In [23]:
#Store_Invoice_No_Day_Service_Name
data['Store_Invoice No_Day_Service']=data['Center Name']+"_"+(data['Invoice No'])+"_"+(data['Day'])+"_"+(data['ServiceName'])

In [24]:
#Total Number ofin a store day wise
groupbytot=data.groupby('Store_Invoice No_Day_Service')['Level'].count()
data['InvoiceNo/Day/Store/Service']=data['Store_Invoice No_Day_Service'].map(groupbytot)

In [25]:
#Total Sale value in a store per month
groupbytot=data.groupby('Center Name')['Sale Value'].sum()
data['totalSL/Month']=data['Center Name'].map(groupbytot)

In [26]:
#Total Sale value in a store average per day
groupbytot=data.groupby('Store_Day')['Sale Value'].sum()
data['totalSV/AvgperDay']=data['Store_Day'].map(groupbytot)
data['totalSV/AvgperDay']=data['totalSV/AvgperDay']/31

In [27]:
#Total Price Paid in a store average per day
groupbytot=data.groupby('Store_Day')['Price Paid'].sum()
data['totalPPD/AvgperDay']=data['Store_Day'].map(groupbytot)
data['totalPPD/AvgperDay']=data['totalPPD/AvgperDay']/31

In [28]:
#Total Price Paid per month in a store 
groupbytot=data.groupby('Center Name')['Price Paid'].sum()
data['totalPPD/Month']=data['Center Name'].map(groupbytot)

In [29]:
data['ClosedOnNA']=data['Closed On'].isna()

In [30]:
data['ClosedOnNA']=data['ClosedOnNA'].astype(int)

In [31]:
data['PromotionNA']=data['Promotion'].isna()
data['PromotionNA']=data['PromotionNA'].astype(int)

In [32]:
#Payment Method
p_m = pd.DataFrame(payment_method, columns={'Payment Method'})

In [33]:
data['Payment Method'] = p_m

<h4> For Checking </h4>

In [4]:
data.columns

In [42]:
#When the Payment Type is null usually the price and discount are the same making net price zero
#but in some cases the net price is not zero even when there is no promotion might be a cancelled transaction
#check[check['Net Price']!=0].iloc[0]

In [43]:
#check['Check'].value_counts()

In [44]:
#data= data.rename({'Sale Value': 'Sale_Value', 'Price Paid': 'Price_Paid'}, axis=1)

In [45]:
#It is weird that when the Sale Value is zero and the Price Paid is zero but there is no discount but still Net Price is zero
#data[(data.Sale_Value==0) & (data.Price_Paid!=0)].iloc[0]

In [46]:
#data= data.rename({'Payment Type': 'Payment_Type'}, axis=1)

In [47]:
#data[(data.Sale_Value==0) & (data.Price_Paid!=0) ]['Payment_Type'].value_counts()

In [48]:
#data[data['Sale_Value']==0][['Payment_Type','Sale_Value','Price_Paid']].iloc[0]

In [49]:
#data[data['Sale_Value']==0].iloc[1]

In [50]:
#data['Payment_Type'].value_counts()

In [51]:
#data[(data.Payment_Type!='Cash') & (data.Payment_Type!='Visa') & (data.Payment_Type!='Mastercard') & (data.Payment_Type!='Debit Card') ]

In [52]:
#data[data['Payment_Type'].isna()].iloc[0]

In [53]:
#data[data['Promotion']=='No Promotion']['Payment_Type'].value_counts()

In [54]:
#data[data['Payment_Type'].isna()].iloc[1]
#['Promotion'].value_counts()

In [55]:
#data['Net Price'].value_counts()
#data['Sale Value'].value_counts()

## Handling null values

In [3]:
data['Sale Value'].value_counts()

In [57]:
#data= data.rename({'Sale Value': 'Sale_Value', 'Price Paid': 'Price_Paid'}, axis=1)

In [1]:
data.isnull().sum()

In [59]:
#Filling null in promotion with No Promotion
data['Promotion'].fillna('No Promotion',inplace=True)

In [2]:
data[(data.Promotion =='No Promotion') & (data.Discount!=0)]

In [61]:
#data['VersionId'].isnull().sum()

In [62]:
#Filling Null Values in Payment Type with NA
#data['Payment_Type'].fillna("NA",inplace=True)

In [63]:
#check=data[data['Payment_Type'].isna()]  

In [64]:
#check['Net Price'].value_counts()

In [65]:
#check['Net Price'].shape

In [66]:
#data= data.rename({'Payment Type': 'Payment_Type', 'Net_Price': 'Net Price'}, axis=1)

In [67]:
#data['Payment Type'] = np.where(data['Net Price'] == 0, data['Cash'] * 0.78125, df['Budget'])

In [68]:
#data['Cashback Redemption'].value_counts()

<h2>Frequency Encoding Categorical Variables</h2>

In [69]:
#Frequency Encoding Center Name
Center_Name=data.groupby('Center Name').size()
Center_Name=Center_Name/len(data)
data['Center Name FE']=data['Center Name'].map(Center_Name)

In [70]:
data['Invoice No']=data['Invoice No'].astype(str)

In [71]:
#Frequency Encoding Invoice No
Invoice_No=data.groupby('Invoice No').size()
Invoice_No=Invoice_No/len(data)
data['Invoice No FE']=data['Invoice No'].map(Invoice_No)

In [72]:
#Service Name
encoding = data.groupby('ServiceName').size()
encoding = encoding/len(data)
services = data['ServiceName'].map(encoding)

In [73]:
data['Services FE'] = services

## One Hot Encoding Categorical Variables with less categories

In [74]:
category=pd.get_dummies(data['Category'],drop_first=True)

In [75]:
data=pd.concat((data,category),axis=1)

In [76]:
Sub_Category=pd.get_dummies(data['Sub Category'],drop_first=True)

In [77]:
data=pd.concat((data,Sub_Category),axis=1)

In [78]:
Group=pd.get_dummies(data['Group'],drop_first=True)
data=pd.concat((data,Group),axis=1)

In [79]:
Proposition=pd.get_dummies(data['Proposition'],drop_first=True)

In [80]:
data=pd.concat((data,Proposition),axis=1)

In [81]:
Level=pd.get_dummies(data['Level'],drop_first=True)
data=pd.concat((data,Level),axis=1)

In [82]:
FocusServices=pd.get_dummies(data['FocusServices'],drop_first=True)
data=pd.concat((data,FocusServices),axis=1)

In [83]:
Status=pd.get_dummies(data['Status'],drop_first=True)
data=pd.concat((data,Status),axis=1)

In [84]:
SubGroup=pd.get_dummies(data['SubGroup'],drop_first=True)
data=pd.concat((data,SubGroup),axis=1)

In [85]:
AppointmentSource=pd.get_dummies(data['AppointmentSource'],drop_first=True)
data=pd.concat((data,AppointmentSource),axis=1)

In [86]:
FirstVisit=pd.get_dummies(data['FirstVisit'],drop_first=True)
data=pd.concat((data,FirstVisit),axis=1)

In [87]:
Member=pd.get_dummies(data['Member'],drop_first=True)
data=pd.concat((data,Member),axis=1)

In [88]:
Requested=pd.get_dummies(data['Requested'],drop_first=True)
data=pd.concat((data,Requested),axis=1)

In [89]:
data['PromotionNA']=data['PromotionNA'].astype(int)

In [90]:
Payment_Method=pd.get_dummies(data['Payment Method'],drop_first=True)
data=pd.concat((data,Payment_Method),axis=1)

In [91]:
Day=pd.get_dummies(data['Day'],drop_first=True)
data=pd.concat((data,Day),axis=1)

<h3>Dropping Values</h3>

<p> Removing Room and Room Category: All Null Values </p>
<p> Removing Start and End Data: Just one repeating value </p>
<p> Removing VersionID : 442346 null values that makes upto 99.98% of the data </p>
<p> Removing GuestCenter: same as Center Name </p>
<p> Membership Service Redemption,'Membership Redemption','Prepaidcard Redemption': All zeros </p>
<p> Receiptno+Center Name="Receipt No"</p>
<p> centerrcptno+Center Name="Invoice No"</p>

In [93]:
data.drop(['Level','FocusServices','Proposition','Sub Category','Category','ServiceCode','Invoice No','Center Code','centerrcptno','Receiptno','Room','Room Category','Start Date','End Date','Promotion'],axis=1,inplace =True)

In [94]:
data.drop(['VersionId','GuestCenter','Membership Service Redemption','Membership Redemption','Prepaidcard Redemption','Cashback Redemption','Serviced On'],axis=1,inplace=True)

In [95]:
data.drop(['Group','SubGroup','Center Name','ServiceId','Receipt No','Sold On'],axis=1,inplace=True)

In [96]:
data.drop(['Closed On','BusinessUnit','AppointmentSource','FirstVisit','Requested','Member'],axis=1,inplace=True)

In [97]:
data.drop(['CreatedBy'],axis=1,inplace=True)

In [99]:
data.drop(['ServiceName','SoldBy'],axis=1,inplace=True)

In [100]:
data.drop(['Payment Type'],axis=1,inplace=True)

In [101]:
data.drop(['Status'],axis=1,inplace=True)

In [102]:
data.drop(['ClosedBy'],axis=1,inplace=True)

In [103]:
data.drop(['Store_Day','Store_Invoice No_Day','Store_Invoice No_Day_Service'],axis=1,inplace=True)

In [104]:
data.drop(['Payment Method'],axis=1,inplace=True)

In [105]:
data.drop(['Day'],axis=1,inplace=True)

In [106]:
#data.select_dtypes(include=['object'])

# Training Isolation Forest to detect anomalies

In [108]:
from sklearn.ensemble import IsolationForest

In [109]:
clf = IsolationForest(max_samples=100, random_state=0)
clf.fit(data)

/home/sonal/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/home/sonal/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


IsolationForest(behaviour='old', bootstrap=False, contamination='legacy',
                max_features=1.0, max_samples=100, n_estimators=100,
                n_jobs=None, random_state=0, verbose=0, warm_start=False)

In [110]:
pred_iso = clf.predict(data)

/home/sonal/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [111]:
Isolation = pd.DataFrame(pred_iso, columns={'Isolation'})

In [112]:
Isolation['Isolation'].value_counts()

 1    398181
-1     44243
Name: Isolation, dtype: int64

In [113]:
Isolation.to_excel('Isolation.xlsx')

In [114]:
data_original['Isolation'] = Isolation

# Training a Variational AutoEncoder (VAE)

In [117]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [119]:
from sklearn.preprocessing import StandardScaler

In [120]:
sc=StandardScaler()

In [121]:
data1=sc.fit_transform(data)

In [123]:
justtest=pd.DataFrame(data1)

In [124]:
justtest["Isolation"]=Isolation['Isolation']

In [125]:
nonoutliers=justtest[justtest["Isolation"]==1]

In [126]:
nonoutliers.drop("Isolation",inplace=True,axis=1)

/home/sonal/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [128]:
X_train, X_test = train_test_split(justtest, test_size=0.2)
X_train = X_train.values
X_test = X_test.values
X_train.shape

(353939, 172)

In [129]:
input_dim = X_train.shape[1]
encoding_dim = 14
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
nb_epoch = 100
batch_size = 32
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error')
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history